In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)

from google.cloud import bigquery
import importlib
import plotly.express as px

import numpy as np
import pandas as pd
from datetime import datetime

import ft_digital_trade.src.utils.read_data_online as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

In [ ]:
client = bigquery.Client()

In [ ]:
#This allows you to select different location of merchant spend for face-to-face and online
## for more information run the following:
##help(read_utils.read_visa)
#visa_data = read_utils.read_f2f_online(cardholder_origin = "all", f2f =  "uk", online =  "all")

#This applies the same filters for face-to-face and online data
## for more information run the following:
##help(read_utils.read_visa)
visa_data_UK = read_utils.read_visa(cardholder_origin = "uk", cardholders_location = "uk", spend_location = "uk")
visa_data_All = read_utils.read_visa(cardholder_origin = "uk", cardholders_location = "uk", spend_location ="all")

In [ ]:
visa_UK = calc_utils.calculate_visa(visa_data_UK)
visa_UK = clean_utils.rename_columns(df = visa_UK, suffix = '_VisaUK')

visa_All = calc_utils.calculate_visa(visa_data_All)
visa_All = clean_utils.rename_columns(df = visa_All, suffix = '_VisaAll')

visa_UK.head()

In [ ]:
global_cards = read_utils.read_global_cards()
global_cards = clean_utils.clean_global(global_cards)
global_cards = calc_utils.calculate_global(global_cards, 'card')

In [ ]:
global_spend = read_utils.read_global_spend()
global_spend = clean_utils.clean_global(global_spend)
global_spend = calc_utils.calculate_global(global_spend, 'spend')

In [ ]:
global_df = global_cards.merge(global_spend, how = 'inner', on = 'year', suffixes = ('_cards', '_spend'))
global_df = clean_utils.rename_columns(df = global_df, suffix = '_global')

In [ ]:
uk_finance = read_utils.read_uk_finance()
uk_finance = clean_utils.clean_uk_finance(uk_finance)
uk_finance = calc_utils.calculate_uk_finance(uk_finance)
uk_finance = uk_finance[['year', 'cardholders','total value of purchases',"total volume of purchases"]]
uk_finance = clean_utils.rename_columns(df = uk_finance , suffix = '_uk_finance')

In [ ]:
boe = read_utils.read_boe()
boe = clean_utils.clean_boe(boe)
boe = calc_utils.calculate_boe(boe)
boe = clean_utils.rename_columns(df = boe , suffix = '_boe')

In [ ]:
link = read_utils.read_link()

In [ ]:
merged_UK = visa_UK.merge(uk_finance, how = 'outer', on = 'year')
merged_UK = merged_UK.merge(boe, how = 'outer', on = 'year')
merged_UK = merged_UK.merge(global_df, how = 'outer', on = 'year')

merged_All = visa_All.merge(uk_finance, how = 'outer', on = 'year')
merged_All = merged_All.merge(boe, how = 'outer', on = 'year')
merged_All = merged_All.merge(global_df, how = 'outer', on = 'year')